In [1]:
import pandas as pd
import re
import numpy as np
import csv
# sklearn packages
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, BayesianRidge
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

RANDOM_SEED = 69

In [2]:
test = pd.read_csv('test.csv',escapechar="\\", quoting=csv.QUOTE_NONE);

In [3]:
train = pd.read_csv('clean_train.csv',escapechar="\\", quoting=csv.QUOTE_NONE);

In [4]:
train.fillna('', inplace=True)

In [4]:
test.fillna('', inplace=True)

In [5]:
train = train[pd.notnull(train["TITLE"]) & pd.notnull(train["DESCRIPTION"]) & pd.notnull(train["BULLET_POINTS"]) & pd.notnull(train["BRAND"])]
# test = test[pd.notnull(test["TITLE"]) & pd.notnull(test["DESCRIPTION"]) & pd.notnull(test["BULLET_POINTS"]) & pd.notnull(test["BRAND"])]

In [6]:
X = train["TITLE"].map(str)+" "+train["DESCRIPTION"].map(str)+" "+train["BULLET_POINTS"].map(str)+" "+train["BRAND"]
y = train["BROWSE_NODE_ID"]

X_test = test["TITLE"].map(str)+" "+test["DESCRIPTION"].map(str)+" "+test["BULLET_POINTS"].map(str)+" "+test["BRAND"]

validation_size = 0.3
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size,
                                                                random_state=RANDOM_SEED)

print("Training Size:- ", int((X.shape[0])*(1-validation_size)))
print("Validation Size:- ", int((X.shape[0])*(validation_size)))
print("Test Size:- ", X_test.shape[0])

Training Size:-  1475141
Validation Size:-  632203
Test Size:-  110775


In [7]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LinearSVC()),
              ])
print(X_train.shape)

nb.fit(X_train[:5000], y_train[:5000])

(1475141,)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC())])

In [12]:
y_pred = nb.predict(X_validation[:10000])
print('Accuracy:-',100*accuracy_score(y_pred, y_validation[:10000]))

Accuracy:- 49.89


In [9]:
y_ans = nb.predict(X_test[:])

In [10]:
for_sub = pd.DataFrame(test['PRODUCT_ID'])
for_sub['BROWSE_NODE_ID']=y_ans
for_sub.to_csv('submission.csv',index=False)

In [11]:
from distributed import Client
import dask as dask

import joblib

client = Client(processes=False)

with joblib.parallel_backend('dask'):
    nb.fit(X_train[:5000], y_train[:5000])